<a href="https://colab.research.google.com/github/shashankbhagat/Recommendation-System/blob/master/Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import csv
from collections import defaultdict
import random
import scipy as sc
import keras as ks
import tensorrec
#!pip install tensorrec
from tensorrec.representation_graphs import (
    AbstractKerasRepresentationGraph, NormalizedLinearRepresentationGraph, LinearRepresentationGraph
)
from tensorrec.util import append_to_string_at_point
import logging
logging.getLogger().setLevel(logging.INFO)

In [4]:
print('Loading Data')
with open('ratings.csv') as ratings_file:
    ratings_file_reader=csv.reader(ratings_file)
    raw_ratings = list(ratings_file_reader)
    raw_ratings_header = raw_ratings.pop(0)

Loading Data


In [0]:
# Iterate through the input to map MovieLens IDs to new internal IDs
# The new internal IDs will be created by the defaultdict on insertion

movielens_to_internal_user_ids=defaultdict(lambda:len(movielens_to_internal_user_ids))
movielens_to_internal_item_ids=defaultdict(lambda:len(movielens_to_internal_item_ids))
for row in raw_ratings:
    row[0] = movielens_to_internal_user_ids[int(row[0])]
    row[1] = movielens_to_internal_item_ids[int(row[1])]
    row[2]=float(row[2])
n_users = len(movielens_to_internal_user_ids)
n_items=len(movielens_to_internal_item_ids)

In [0]:
random.shuffle(raw_ratings)
cutoff=int(0.8*len(raw_ratings))
train_ratings=raw_ratings[:cutoff]
test_ratings=raw_ratings[cutoff:]

In [0]:
# This method converts a list of (user, item, rating, time) to a sparse matrix
def interactions_list_to_sparse_matrix(interactions):
    user_column,item_column, ratings_column, _ = zip(*interactions)
    return sc.sparse.coo_matrix((ratings_column,(user_column,item_column)),shape=(n_users,n_items))

# Create sparse matrices of interaction data
sparse_train_ratings = interactions_list_to_sparse_matrix(train_ratings)
sparse_test_ratings = interactions_list_to_sparse_matrix(test_ratings)

In [9]:
# Construct indicator features for users and items
user_indicator_features = sc.sparse.identity(n_users)
item_indicator_features = sc.sparse.identity(n_items)

# Build a matrix factorization collaborative filter model
cf_model = tensorrec.TensorRec(n_components=5)

#Fit the collaborative filter model
print('Training collaborative filter')
cf_model.fit(interactions=sparse_train_ratings,
             user_features=user_indicator_features, 
             item_features=item_indicator_features)

Training collaborative filter


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [15]:
# Create sets of train/test interactions that are only ratings >= 4.0
sparse_train_ratings_4plus=sparse_train_ratings.multiply(sparse_train_ratings>=4.0)
sparse_test_ratings_4plus=sparse_test_ratings.multiply(sparse_test_ratings>=4.0)

# This method consumes item ranks for each user and prints out recall@10 train/test metrics
def check_results(ranks):
    train_recall_at_10 = tensorrec.eval.recall_at_k(test_interactions=sparse_test_ratings_4plus,
                                                    predicted_ranks=ranks,
                                                   k=10).mean()
    test_recall_at_10 = tensorrec.eval.recall_at_k(test_interactions=sparse_train_ratings_4plus,
                                                    predicted_ranks=ranks,
                                                   k=10).mean()
    
    print("Recall at 10: Train: {:.4f} Test: {:.4f}".format(train_recall_at_10,test_recall_at_10))
    
#Check result of CF
print("Matrix fatorization collaborative filter:")
predicted_ranks = cf_model.predict_rank(user_features=user_indicator_features,
                                           item_features = item_indicator_features)
check_results(predicted_ranks)    

Matrix fatorization collaborative filter:
Recall at 10: Train: 0.0017 Test: 0.0008


In [43]:
#Another loss function: WMRB
print('Training collaborative filter with WMRB loss')
ranking_cf_model = tensorrec.TensorRec(n_components=5,loss_graph=tensorrec.loss_graphs.WMRBLossGraph())
log_dir="/content/sample_data/log/"

ranking_cf_model.fit(interactions=sparse_train_ratings_4plus, user_features=user_indicator_features, item_features=item_indicator_features, n_sampled_items=int(n_items*.01))
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1,write_graph=True, write_images=True)


#Check results of the WMRB Matrix Factorization CF model
predicted_ranks = ranking_cf_model.predict_rank(user_features=user_indicator_features, 
                                                item_features=item_indicator_features)
check_results(predicted_ranks)


Training collaborative filter with WMRB loss


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Recall at 10: Train: 0.0666 Test: 0.1070


In [73]:
!tensorboard --logdir log_dir


TensorBoard 1.15.0a20190901 at http://a24e18582ab1:6006/ (Press CTRL+C to quit)


In [0]:
# Construct a Keras representation graph by inheriting tensorrec.representation_graphs.AbstractKerasRepresentationGraph
class DeepRepresentationGraph(AbstractKerasRepresentationGraph):

    # This method returns an ordered list of Keras layers connecting the user/item features to the user/item
    # representation. When TensorRec learns, the learning will happen in these layers.
    def create_layers(self, n_features, n_components):
        return [
            ks.layers.Dense(n_components * 16, activation='relu'),
            ks.layers.Dense(n_components * 8, activation='relu'),
            ks.layers.Dense(n_components * 2, activation='relu'),
            ks.layers.Dense(n_components, activation='tanh'),
        ]


In [90]:
# Build results header string
result_strings = []
header = "UserRepr Graph"
header = append_to_string_at_point(header, 'ItemRepr Graph', 40)
header = append_to_string_at_point(header, 'Rec. In-Sample', 70)
header = append_to_string_at_point(header, 'Rec. Out-sample', 90)
header = append_to_string_at_point(header, 'Prec. In-Sample', 110)
header = append_to_string_at_point(header, 'Prec. Out-sample', 130)
header = append_to_string_at_point(header, 'NDCG In-Sample', 150)
header = append_to_string_at_point(header, 'NDCG Out-sample', 170)
result_strings.append(header)

for user_repr in (NormalizedLinearRepresentationGraph, DeepRepresentationGraph):
    for item_repr in (LinearRepresentationGraph, DeepRepresentationGraph):
        model = tensorrec.TensorRec(n_components=20,
                          item_repr_graph=item_repr(),
                          user_repr_graph=user_repr(),
                          loss_graph=tensorrec.loss_graphs.BalancedWMRBLossGraph(),
                          biased=False)
        
        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1,write_graph=True, write_images=True)
        #tensorboard_callback.set_model(model)

        # Fit the model and get a result packet
        fit_kwargs = {'epochs': 5, 'learning_rate': .01, 'n_sampled_items': 100, 'verbose': True}
        result = tensorrec.eval.fit_and_eval(model, user_indicator_features, item_indicator_features, sparse_train_ratings_4plus, sparse_test_ratings_4plus, fit_kwargs,
                              recall_k=100, precision_k=100, ndcg_k=100)
        
        # Build results row for this configuration
        res_string = "{}".format(user_repr.__name__)
        res_string = append_to_string_at_point(res_string, item_repr.__name__, 40)
        res_string = append_to_string_at_point(res_string, ": {:0.4f}".format(result[3]), 68)
        res_string = append_to_string_at_point(res_string, "{:0.4f}".format(result[0]), 90)
        res_string = append_to_string_at_point(res_string, "{:0.4f}".format(result[4]), 110)
        res_string = append_to_string_at_point(res_string, "{:0.4f}".format(result[1]), 130)
        res_string = append_to_string_at_point(res_string, "{:0.4f}".format(result[5]), 150)
        res_string = append_to_string_at_point(res_string, "{:0.4f}".format(result[2]), 170)
        logging.info(header)
        logging.info(res_string)
        result_strings.append(res_string)

# Log the final results of all models
for res_string in result_strings:
    logging.info(res_string)

I0902 08:53:06.913445 139946425325440 tensorrec.py:587] Processing interaction and feature data
/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
I0902 08:53:14.508519 139946425325440 tensorrec.py:615] Beginning fitting
I0902 08:53:15.750882 139946425325440 tensorrec.py:633] EPOCH 0 BATCH 0 loss = 6.277060031890869, weight_reg_l2_loss = 0.05167000000000001, mean_pred = 0.0009051727829501033
I0902 08:53:16.011991 139946425325440 tensorrec.py:633] EPOCH 1 BATCH 0 loss = 6.265364646911621, weight_reg_l2_loss = 0.0512474609375, mean_pred = 0.011366836726665497
I0902 08:53:16.275081 139946425325440 tensorrec.py:633] EPOCH 2 BATCH 0 loss = 6.251622676849365, weight_reg_l2_loss = 0.050943774414062504, mean_pred = 0.021836882457137108
I0902 08:53:16.537211 13994642532

In [91]:
!tensorboard --logdir log_dir

TensorBoard 1.15.0a20190901 at http://a24e18582ab1:6006/ (Press CTRL+C to quit)
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/werkzeug/serving.py", line 734, in serve_forever
    HTTPServer.serve_forever(self)
  File "/usr/lib/python3.6/socketserver.py", line 236, in serve_forever
    ready = selector.select(poll_interval)
  File "/usr/lib/python3.6/selectors.py", line 376, in select
    fd_event_list = self._poll.poll(timeout)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/bin/tensorboard", line 10, in <module>
    sys.exit(run_main())
  File "/usr/local/lib/python3.6/dist-packages/tensorboard/main.py", line 64, in run_main
    app.run(tensorboard.main, flags_parser=tensorboard.configure)
  File "/usr/local/lib/python3.6/dist-packages/absl/app.py", line 300, in run
    _run_main(main, args)
  File "/usr/local/lib/python3.6/dist-packages/absl/app.py", line 

TypeError: ignored